Import Libraries and Initialize Constants

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
from scipy.special import expit
import os
import json

MODEL = "cardiffnlp/tweet-topic-21-multi"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
class_mapping = model.config.id2label

/home/neonhydra64/mambaforge/envs/GCRL2000Ens/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Convert Data

In [4]:
# Prepare your full dataset (list of input texts)
folds= ['charliehebdo-all-rnr-threads','ottawashooting-all-rnr-threads',
'ebola-essien-all-rnr-threads','prince-toronto-all-rnr-threads',
'ferguson-all-rnr-threads',	'putinmissing-all-rnr-threads',
'germanwings-crash-all-rnr-threads',
'gurlitt-all-rnr-threads','sydneysiege-all-rnr-threads']

full_dataset = []
y=[]

# Open the file in read mode
with open('source_tweets.txt', 'r') as file:
    # Read each line in the file
    for line in file:
        # Split the line using the tab character to separate the id and sentence
        _, sentence = line.strip().split('\t')
        
        # Remove the word "URL" from the sentence
        sentence = sentence.replace('URL', '')
        
        # Append the cleaned sentence to the array
        full_dataset.append(sentence)


In [5]:
print(full_dataset[0])
print(full_dataset[1])
print(full_dataset[3])

🔥ca kkk grand wizard 🔥 endorses @hillaryclinton #neverhillary #trump2016 URL
an open letter to trump voters from his top strategist-turned-defector URL via @xojanedotcom
brandon marshall visits and offers advice, support to brother of fallen hero zaevion dobson: URL URL


Prepare and Classify the Dataset

In [6]:
# Create lists to store results
texts = []
predictions = []
confidence_scores = []
rumour_count = 0

# Create dictionaries to store class counts and confidence sums
class_counts = {class_label: 0 for class_label in class_mapping.values()}
confidence_sums = {class_label: 0.0 for class_label in class_mapping.values()}

# Iterate through the dataset and classify each text
for text in full_dataset:
    rumour_count += 1
    print("rumour: " + str(rumour_count))
    tokens = tokenizer(text, return_tensors="pt")
    output = model(**tokens)

    scores = output.logits[0].detach().numpy()
    scores = expit(scores)
    highest_score_index = np.argmax(scores)
    class_label = class_mapping[highest_score_index]
    confidence_score = scores[highest_score_index]

    class_counts[class_label] += 1
    confidence_sums[class_label] += np.max(scores)

    texts.append(text)
    predictions.append(class_label)
    confidence_scores.append(confidence_score)


rumour: 1


rumour: 2
rumour: 3
rumour: 4
rumour: 5
rumour: 6
rumour: 7
rumour: 8
rumour: 9
rumour: 10
rumour: 11
rumour: 12
rumour: 13
rumour: 14
rumour: 15
rumour: 16
rumour: 17
rumour: 18
rumour: 19
rumour: 20
rumour: 21
rumour: 22
rumour: 23
rumour: 24
rumour: 25
rumour: 26
rumour: 27
rumour: 28
rumour: 29
rumour: 30
rumour: 31
rumour: 32
rumour: 33
rumour: 34
rumour: 35
rumour: 36
rumour: 37
rumour: 38
rumour: 39
rumour: 40
rumour: 41
rumour: 42
rumour: 43
rumour: 44
rumour: 45
rumour: 46
rumour: 47
rumour: 48
rumour: 49
rumour: 50
rumour: 51
rumour: 52
rumour: 53
rumour: 54
rumour: 55
rumour: 56
rumour: 57
rumour: 58
rumour: 59
rumour: 60
rumour: 61
rumour: 62
rumour: 63
rumour: 64
rumour: 65
rumour: 66
rumour: 67
rumour: 68
rumour: 69
rumour: 70
rumour: 71
rumour: 72
rumour: 73
rumour: 74
rumour: 75
rumour: 76
rumour: 77
rumour: 78
rumour: 79
rumour: 80
rumour: 81
rumour: 82
rumour: 83
rumour: 84
rumour: 85
rumour: 86
rumour: 87
rumour: 88
rumour: 89
rumour: 90
rumour: 91
rumour: 92
rumour:

Save Results to CSV

In [7]:
# Save the results to a file (e.g., CSV)
import csv

output_file = "classification_results.csv"
with open(output_file, "w", newline="") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Text", "Predicted_Class", "Confidence_Score"])
    for text, pred, score in zip(texts, predictions, confidence_scores):
        csvwriter.writerow([text, pred, score])

print("Classification results saved to:", output_file)


Classification results saved to: classification_results.csv


Calculate and Display Percentages

In [8]:
# Calculate percentages
total_texts = len(full_dataset)
class_percentages = {
  class_label: count / total_texts * 100 for class_label, count in class_counts.items()
}

# Display the percentages
print("Percentage of texts classified into each class:")
for class_label, percentage in class_percentages.items():
  print(f"{class_label}: {percentage:.2f}%")

Percentage of texts classified into each class:
arts_&_culture: 0.40%
business_&_entrepreneurs: 2.28%
celebrity_&_pop_culture: 8.19%
diaries_&_daily_life: 4.50%
family: 0.00%
fashion_&_style: 0.13%
film_tv_&_video: 3.49%
fitness_&_health: 0.67%
food_&_dining: 2.75%
gaming: 0.20%
learning_&_educational: 1.01%
music: 1.07%
news_&_social_concern: 67.05%
other_hobbies: 0.20%
relationships: 0.13%
science_&_technology: 2.55%
sports: 4.70%
travel_&_adventure: 0.60%
youth_&_student_life: 0.07%


In [1]:
defualt_rules = {
    "bathroom_light" : 100,
    "bathroom_temperature" : [20, 25],
    "home_temperature" : [20, 25],
    "bathroom_activity" : 20,
    "daytime_inactivity" : 60,
    "nighttime_inactivity" : 240,
    "sensor_battery" : 50
}

for key, value in defualt_rules.items():
    rule = {key : value}
    print(rule)

{'bathroom_light': 100}
{'bathroom_temperature': [20, 25]}
{'home_temperature': [20, 25]}
{'bathroom_activity': 20}
{'daytime_inactivity': 60}
{'nighttime_inactivity': 240}
{'sensor_battery': 50}
